In [ ]:
using Pkg; Pkg.activate(joinpath(@__DIR__, ".."))
using LinearAlgebra
using ForwardDiff
using Plots
using HybridRobotDynamics
using SiLQR
using ImpactSafety

In [ ]:
"""
Set up system and controller
"""

# Crazyflie 2.1 parameters: https://arxiv.org/pdf/1608.05786
e = 1.0
g = 9.81
m = 0.027
j = [1.436e-5, 1.395e-5,  2.173e-5]
b = 7.9379e-12 / 3.1582e-10 * ones(4)
c = 0.0283 * ones(4)
d = 0.0283 * ones(4)
sys = bouncing_quadrotor(e, g, m, j, b, c, d)

# Stage and terminal costs
Q = 1e-6 * diagm([2e2*ones(3); 1e2*ones(4); ones(6)])
R = 1e-9 * I(sys.nu)
Qf = 1e2 * Q
stage(x, u) = x'*Q*x + u'*R*u
terminal(x) = x'*Qf*x

# RK4 integrator
rk4 = ExplicitIntegrator(:rk4)

# Problem parameters
N = 50
Δt = 0.025
params = SiLQR.Parameters(sys, stage, terminal, rk4, N, Δt)

# Reference trajectory
pref = [10.0, 10.0, 4.0]
qref = [1.0; zeros(3)]
xref = [pref; qref; zeros(6)]
uref = zeros(sys.nu)
params.xrefs = [xref for k = 1:N]
params.urefs = [uref for k = 1:(N-1)]

# Initial conditions
p0 = [0.0, 0.0, 4.0]
q0 = qref #random_unit_quat()
v0 = [7.0, 7.0, -2.0]
ω0 = zeros(3)
params.x0 = [p0; q0; v0; ω0]
params.mI = :flight

nothing

In [ ]:
"""
Solve using SiLQR
"""

# Solve
sol = SiLQR.Solution(params)
cache = SiLQR.Cache(params)
@time SiLQR.solve!(sol, cache, params; max_iter=200, max_ls_iter=20, stat_tol=1e-6)

# Visualize states
xs = reduce(vcat, sol.xs)
plot_2d_states(N, sys.nx, (1,3), xs; xlim=(0,10), ylim=(0,10))
plot_2d_states(N, sys.nx, (1,2), xs; xlim=(0,10), ylim=(0,10))

nothing

In [ ]:
function post_impact_velocity(
    flow::Union{ControlAffineFlow, Function},
    trn::Transition,
    vidx::UnitRange{Int},
    x::Vector,
    u::Vector,
    Δt::Vector
)::Vector
    x1 = x + Δt*flow(x, u)
    xJ = trn.reset(x1)
    return xJ[vidx]
end

In [ ]:
"""
Obstacle avoidance auxiliary functions
"""

function opos(o::Vector)::Vector
    return o[1:3]
end

function ovel(o::Vector)::Vector
    return o[4:6]
end

function xpos(x::Vector)::Vector
    return x[1:3]
end

function xvel(x::Vector)::Vector
    return x[8:10]
end

function decoupled_Φ(n::Real, k::Real, dmin::Real, o::Vector, p::Vector, v::Vector)
    rel_dist = opos(o) - p
    d = norm(rel_dist)
    d_unit = rel_dist / d
    rel_vel = ovel(o) - v
    ḋ = rel_vel' * d_unit
    return dmin*n .- d*n .- k*ḋ
end

function Φ(n::Real, k::Real, dmin::Real, o::Vector, x::Vector)::Vector
    #@show n, k, dmin
    rel_dist = opos(o) - xpos(x)
    d = norm(rel_dist)
    d_unit = rel_dist / d
    rel_vel = ovel(o) - xvel(x)
    ḋ = rel_vel' * d_unit
    return [dmin*n - d*n - k*ḋ]
end

function Φ̇ssa(Φargs::Tuple, η::Real, x::Vector)::Vector
    Φval = Φ(Φargs..., x)
    if Φval[1] > 1e-9
        return [-η]
    elseif Φval[1] > -1e-9
        return [0.0]
    else
        return [Inf]
    end
end

function Φ̇cbf(Φargs::Tuple, λ::Real, x::Vector)::Vector
    Φval = Φ(Φargs..., x)
    return -λ*Φval
end

nothing

In [ ]:
"""
Transition functions that are compatible with non-DiffFloats
"""

function guard(x::Vector)
    return x[3]
end

function elastic_impact(x::Vector)::Vector
    return [x[1:2]; 1e-9; x[4:9]; -x[10]; x[11:13]]
end

nothing

In [ ]:
"""
Set up safety filter
"""

# Safety index params
n = 1
κ = 0.5
dmin = 1

# cbf params
λ = 1e2

# Safety filter
ϵ = 0.0
cbf = SafetyFilter(sys.nu, ϵ, Φ, Φ̇cbf, sys.modes[:flight].flow)

# Obstacle state
o = [5, 5, 2, 0.0, 0.0, 0.0]

nothing


In [ ]:
"""
Simulate the impact-unaware combined policy
"""

# Sim settings
speedup = 10
Δtsim = Δt / speedup
Nsim = 1 + (N-1)*speedup

# Timing mapping from sim to TVLQR
kmap = length(cache.bwd.ds) / Nsim

# Initial conditions
xs_sim = [zeros(sys.nx) for k = 1:Nsim]
us_sim = [zeros(sys.nu) for k = 1:(Nsim-1)]
xs_sim[1] = sol.xs[1]
mI = sys.modes[:flight]

# Simulate
try
for k = 1 : (Nsim-1)
    # Nominal controller
    klqr = trunc(Int, 1 + k*kmap)
    unom = sol.us[klqr]
    # Filtered controller
    Φargs = (n, κ, dmin, o)
    Φ̇args = (Φargs, λ)
    us_sim[k] = cbf(Φargs, Φ̇args, xs_sim[k], unom; verbose=false)

    Φval = Φ(Φargs..., xs_sim[k])
    if Φval[1] > 0.0
        @show xs_sim[k][1], Φval[1]
    end

    # Integrate smooth dynamics
    xs_sim[k+1] = rk4(mI.flow, xs_sim[k], us_sim[k], Δtsim)

    # Reset and update mode if guard is hit
    for (trans, mJ) in mI.transitions
        if trans.guard(xs_sim[k+1]) < 0.0
            xs_sim[k+1] = trans.reset(xs_sim[k+1])
            mI = mJ
            break
        end
    end
end
catch e
end

# Visualize
plot_2d_states(Nsim, sys.nx, (1,3), vcat(xs_sim...))
plot_2d_states(Nsim, sys.nx, (1,2), vcat(xs_sim...))

# Create spherical coordinates
θ = range(0, 2π, length=10)
ϕ = range(0, π, length=10)
sx = reshape([o[1] + dmin*sin(ϕi)*cos(θj) for ϕi in ϕ, θj in θ], 100)
sy = reshape([o[2] + dmin*sin(ϕi)*sin(θj) for ϕi in ϕ, θj in θ], 100)
sz = reshape([o[3] + dmin*cos(ϕi) for ϕi in ϕ, θj in θ], 100)

px = []
py = []
pz = []

# Animate
gr()
p = scatter(sx, sy, sz, color = :red, legend=false)
anim = @animate for k = 1:10:Nsim
    push!(px, xs_sim[k][1])
    push!(py, xs_sim[k][2])
    push!(pz, xs_sim[k][3])
    plot3d!(p, px, py, pz, color=:blue, marker=false, xlim=(0,10), ylim=(0,10), zlim=(0,10))
end
gif(anim, "unsafe.gif", fps=20)

nothing

In [ ]:
"""
Simulate the combined impact-aware policy
"""

# Impact-aware safety filter
q = 1e2
ρ = 0
pidx = 1:3
vidx = 8:10
iacbf = ImpactAwareSafetyFilter(
    q, ρ, Δtsim, pidx, vidx,
    guard, elastic_impact, decoupled_Φ, cbf
)

# Initial conditions
xs_sim = [zeros(sys.nx) for k = 1:Nsim]
us_sim = [zeros(sys.nu) for k = 1:(Nsim-1)]
xs_sim[1] = sol.xs[1]
mI = sys.modes[:flight]

# Simulate
try
for k = 1 : (Nsim-1)
    # Nominal controller
    klqr = trunc(Int, 1 + k*kmap)
    unom = sol.us[klqr] - cache.bwd.Ks[klqr] * (xs_sim[k] - sol.xs[klqr])

    # Filtered controller
    Φargs = (n, κ, dmin, o)
    Φ̇args = (Φargs, λ)
    us_sim[k] = iacbf(Φargs, Φ̇args, xs_sim[k], unom; verbose=false)

    Φval = Φ(Φargs..., xs_sim[k])
    if Φval[1] > 0.0
        println("k:", k, " Φ:", Φval[1])
    end


    # Integrate smooth dynamics
    xs_sim[k+1] = rk4(mI.flow, xs_sim[k], us_sim[k], Δtsim)

    # Reset and update mode if guard is hit
    for (trans, mJ) in mI.transitions
        if trans.guard(xs_sim[k+1]) < 0.0
            xs_sim[k+1] = trans.reset(xs_sim[k+1])
            mI = mJ
            break
        end
    end
end
catch e
    @show e
end

# Visualize
plot_2d_states(Nsim, sys.nx, (1,3), vcat(xs_sim...))
plot_2d_states(Nsim, sys.nx, (1,2), vcat(xs_sim...))


px = []
py = []
pz = []

# Animate
gr()
p = scatter(sx, sy, sz, color=:red, legend=false)
anim = @animate for k = 1:10:Nsim
    push!(px, xs_sim[k][1])
    push!(py, xs_sim[k][2])
    push!(pz, xs_sim[k][3])
    plot3d!(p, px, py, pz, color=:blue, marker=false, xlim=(0,10), ylim=(0,10), zlim=(0,10))
end
gif(anim, "safe.gif", fps=20)

nothing